# archive to folder

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "yjbds/ml-base:0.4.8:"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'yjbds/ml-base:0.4.8:'


In [3]:
import os
import zipfile
import urllib.request
import tarfile
import json

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

from typing import Union

def open_archive(
    context: MLClientCtx, 
    archive_url: DataItem,
    subdir: str = "content",
    key: str = "content"
):
    """Open a file/object archive into a target directory
    
    Currently supports zip and tar.gz
    
    :param context:      function execution context
    :param archive_url:  url of archive file
    :param subdir:       path within artifact store where extracted files
                         are stored
    :param key:          key of archive contents in artifact store
    """
    os.makedirs(subdir, exist_ok=True)
    
    archive_url = archive_url.local()
    if archive_url.endswith("gz"):
        with tarfile.open(archive_url, mode="r|gz") as ref:
            ref.extractall(subdir)
    elif archive_url.endswith("zip"):
        with zipfile.ZipFile(archive_url, "r") as ref:
            ref.extractall(subdir)
    else:
        raise ValueError(f'unsupported archive type in {archive_url}')
    
    context.log_artifact(key, local_path=subdir)

In [4]:
# nuclio: end-code

### mlconfig

In [5]:
from mlrun import mlconf
import os

mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [6]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("open_archive")

# add metadata (for templates and reuse)
fn.spec.default_handler = "open_archive"
fn.spec.description = "Open a file/object archive into a target directory"
fn.metadata.categories = ["data-movement", "utils"]
fn.metadata.labels = {"author": "yaronh"}
fn.export("function.yaml")

[mlrun] 2020-05-25 01:46:04,755 function spec saved to path: function.yaml


In [12]:
from mlutils import get_vol_mount
fn.apply(get_vol_mount())

## tests

In [9]:
from mlrun import run_local

### tar

In [10]:
run = run_local(
    handler=open_archive, 
    inputs={'archive_url': "https://fpsignals-public.s3.amazonaws.com/catsndogs.tar.gz"})

[mlrun] 2020-05-25 01:46:10,846 starting run mlrun-089735-open_archive uid=3d6807658e224a50bd45b2673140b7ba  -> http://mlrun-api:8080
[mlrun] 2020-05-25 01:46:11,179 downloading https://fpsignals-public.s3.amazonaws.com/catsndogs.tar.gz to local tmp
[mlrun] 2020-05-25 01:49:38,798 log artifact content at /home/jovyan/data/content/, size: None, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...40b7ba,0,May 25 01:46:11,completed,mlrun-089735-open_archive,kind=handlerowner=jovyanhost=jupyter-notebook-6447b7f894-m4vj4,archive_url,,,content


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3d6807658e224a50bd45b2673140b7ba --project default , !mlrun logs 3d6807658e224a50bd45b2673140b7ba --project default
[mlrun] 2020-05-25 01:49:38,841 run executed, status=completed


### zip

In [11]:
run_local(
    handler=open_archive, 
    inputs={'archive_url': 'http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip'})

[mlrun] 2020-05-25 01:49:38,855 starting run mlrun-89ded3-open_archive uid=f6c19c20dcb2438a85dd536c55c3b885  -> http://mlrun-api:8080
[mlrun] 2020-05-25 01:49:39,382 downloading http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip to local tmp
[mlrun] 2020-05-25 01:50:32,028 log artifact content at /home/jovyan/data/content/, size: None, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...c3b885,0,May 25 01:49:38,completed,mlrun-89ded3-open_archive,kind=handlerowner=jovyanhost=jupyter-notebook-6447b7f894-m4vj4,archive_url,,,content


to track results use .show() or .logs() or in CLI: 
!mlrun get run f6c19c20dcb2438a85dd536c55c3b885 --project default , !mlrun logs f6c19c20dcb2438a85dd536c55c3b885 --project default
[mlrun] 2020-05-25 01:50:32,051 run executed, status=completed
